on doit extraire les notes des films et savoir si et ou ils sont disponibles


In [ ]:
import os
from dotenv import load_dotenv
import requests
import json
import polars as pl
import time
from pathlib import Path

from functools import wraps
# Import complet
import urllib.request
import urllib.parse
import urllib.error


### Load les api_key saved in .env (not kept in git)


In [2]:
load_dotenv()  # This loads the variables from .env into the environment


True

In [3]:
from api_counter import PersistentAPICounter, count_api_decorator


In [4]:

# Option 1: Default name
counter = PersistentAPICounter("api_couter.json")  # Uses 'api_usage.json'
count_api = count_api_decorator(counter)


📝 File api_couter.json not found, creating new one...
✓ Created api_couter.json


In [5]:
@count_api('Watchmode')  # ← Compte Watchmode
def watchmode_info(release_date_start, release_date_end, api_key, page):

    # --- Configuration ---

    base_url = "https://api.watchmode.com/v1/list-titles/"
    params = {
        "apiKey": api_key,
        "types": "movie",
        "regions": "CA",
        "release_date_start": release_date_start,
        "release_date_end": release_date_end,
        "limit": "250",
        "sort_by": "release_date_desc"
    }

    # --- Construire la requête ---
    query = "&".join(f"{k}={v}" for k, v in params.items())

    url = f"{base_url}?{query}&page={page}"
    print(f"📄 Chargement page {page}...")

    try:
        with urllib.request.urlopen(url) as response:
            data = json.loads(response.read().decode())
    except Exception as e:
         print(f"❌ Erreur à la page {page}: {e}")

    titles = data.get("titles", [])
    if not titles:
        print("✅ Fin des résultats.")


    
    # --- Résumé ---
    print(f"\n🎉 Total récupéré : {len(titles)} films.\n")
    return titles


In [6]:

@count_api('OMDB')  # ← Compte OMDB

def get_movie_info(imdb_id, api_key):
    """Récupère metascore, genre et plot avec gestion robuste des NA"""
    
    # Gérer les imdb_id nuls/invalides
    if not imdb_id or imdb_id == 'N/A':
        return {'metascore': None, 'genre': None, 'plot': None}
    
    url = f"http://www.omdbapi.com/?i={imdb_id}&apikey={api_key}"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        movie_data = response.json()
        
        if movie_data.get("Response") == "True":
            # Convertir 'N/A' en None et nettoyer
            metascore = movie_data.get('Metascore', 'N/A')
            
            genre = movie_data.get('Genre','N/A')
            
            plot = movie_data.get('Plot','N/A')
            
            return {
                'metascore': metascore,
                'genre': genre,
                'plot': plot
            }
        else:
            # Film non trouvé dans OMDb
            print(f"⚠ Film non trouvé: {imdb_id}")
            return {'metascore': None, 'genre': None, 'plot': None}
            
    except requests.exceptions.Timeout:
        print(f"⏱ Timeout: {imdb_id}")
        return {'metascore': None, 'genre': None, 'plot': None}
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Erreur API pour {imdb_id}: {e}")
        return {'metascore': None, 'genre': None, 'plot': None}
        
    except Exception as e:
        print(f"❌ Erreur inattendue pour {imdb_id}: {e}")
        return {'metascore': None, 'genre': None, 'plot': None}




In [7]:
@count_api('Watchmode')  # ← Compte Watchmode

def get_sources_info(watchmode_id,API_KEY_watchmode):
    """Récupère et analyse les sources pour un film"""
    
    if not watchmode_id or watchmode_id is None:
        return {'nb_sources': None, 'sources': None, 'prix_min': None, 'sources_min': None}
    
    url = f'https://api.watchmode.com/v1/title/{watchmode_id}/sources/?apiKey={API_KEY_watchmode}&regions=CA'
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        movie_data = response.json()
        
        # Analyser les sources (un seul parcours)
        sources_uniques = set()
        prix_min = float('inf')
        sources_min = set()
        
        for s in movie_data:
            sources_uniques.add(s['name'])
            
            price = s.get('price')
            if price is not None:
                if price < prix_min:
                    prix_min = price
                    sources_min = {s['name']}
                elif price == prix_min:
                    sources_min.add(s['name'])
        
        return {
            'nb_sources': len(movie_data),
            'sources': list(sources_uniques),
            'prix_min': prix_min if prix_min != float('inf') else None,
            'sources_min': list(sources_min) if sources_min else None
        }
        
    except Exception as e:
        print(f"❌ Erreur pour watchmode_id {watchmode_id}: {e}")
        return {'nb_sources': None, 'sources': None, 'prix_min': None, 'sources_min': None}





In [8]:
def final(api_watchmode, api_omdb, page, file_save):

    all_titles = watchmode_info('20250101','20251231',api_watchmode,page)
    movies_df = pl.DataFrame(all_titles)
    movies_df = movies_df.filter(pl.col("type") == "movie")
    movies_df
    # Optional: also remove empty strings if they exist
    movies_df = movies_df.filter(
        (pl.col("imdb_id").is_not_null()) & 
        (pl.col("imdb_id") != "")
    )

    # Check how many rows remain
    print(f"Rows after cleaning: {len(movies_df)}")

    movies_df = movies_df.select([
        pl.col("id").alias("watchmode_id"),
        pl.col("imdb_id"),
        pl.col("title"),
        pl.col("year")])
    movies_df


    # Appliquer avec gestion des erreurs
    print("🔍 Récupération des infos OMDb...")

    movies_df = movies_df.with_columns(
        pl.col("imdb_id").map_elements(
            lambda x: (get_movie_info(x, api_key=api_omdb), time.sleep(0.2))[0],  # Rate limit: 5 req/sec
            return_dtype=pl.Struct({
                'metascore': pl.Utf8,  # Garder en string pour éviter erreurs de conversion
                'genre': pl.Utf8,
                'plot': pl.Utf8
            })
        ).alias("movie_info")
    ).unnest("movie_info")
    # Après avoir créé les colonnes avec unnest
    movies_df = movies_df.with_columns(
    pl.col('metascore').cast(pl.Int32, strict=False)
    )

    movies_df
    movies_df = movies_df.filter(pl.col('metascore').is_not_null())
    print(len(movies_df))
    movies_df = movies_df.with_columns(
    pl.col("watchmode_id").map_elements(
        lambda x: (get_sources_info(x,api_watchmode ), time.sleep(0.2))[0],  # Rate limit
        return_dtype=pl.Struct({
            'nb_sources': pl.Int64,
            'sources': pl.List(pl.Utf8),
            'prix_min': pl.Float64,
            'sources_min': pl.List(pl.Utf8)
        })
    ).alias("sources_info")
    ).unnest("sources_info")

    # Vérifier
    print(movies_df.select(['watchmode_id', 'nb_sources', 'sources', 'prix_min', 'sources_min']).head())
    len(movies_df)
    movies_df.write_parquet(f'{file_save}.parquet')





In [11]:
api_key = os.getenv("API_KEY_omdbapi_6")
API_KEY_watchmode = os.getenv("API_KEY_watchmode_6")


In [ ]:
for page in range(1,5):
    final(API_KEY_watchmode, api_key, page, f"movies_{page}")


In [12]:
for page in range(4,5):
    final(API_KEY_watchmode, api_key, page, f"movies_{page}")


📄 Chargement page 4...

🎉 Total récupéré : 142 films.

Rows after cleaning: 142
🔍 Récupération des infos OMDb...
34
shape: (5, 5)
┌──────────────┬────────────┬─────────────────────────┬──────────┬─────────────────────────────────┐
│ watchmode_id ┆ nb_sources ┆ sources                 ┆ prix_min ┆ sources_min                     │
│ ---          ┆ ---        ┆ ---                     ┆ ---      ┆ ---                             │
│ i64          ┆ i64        ┆ list[str]               ┆ f64      ┆ list[str]                       │
╞══════════════╪════════════╪═════════════════════════╪══════════╪═════════════════════════════════╡
│ 1796397      ┆ 1          ┆ ["Netflix"]             ┆ null     ┆ null                            │
│ 1633704      ┆ 16         ┆ ["Amazon", "AppleTV", … ┆ 6.99     ┆ ["Amazon", "AppleTV", "Cineple… │
│              ┆            ┆ "YouTu…                 ┆          ┆                                 │
│ 1762467      ┆ 1          ┆ ["Netflix"]             ┆ null  

In [13]:
counter.show()


API USAGE SUMMARY
  OMDB: 1034 calls
  Watchmode: 251 calls
----------------------------------------
  TOTAL: 1285 calls



In [ ]:
import polars as pl

# Lire et combiner
movies_update_df = pl.concat([
    pl.read_parquet('movies_1.parquet'),
    pl.read_parquet('movies_2.parquet'),
    pl.read_parquet('movies_3.parquet'),
    pl.read_parquet('movies_4.parquet')
])

print(f"Avant dédoublonnage: {len(movies_update_df)} films")

# Supprimer les doublons (par imdb_id ou watchmode_id)
movies_update_df = movies_update_df.unique(subset=['imdb_id'])

print(f"Après dédoublonnage: {len(movies_update_df)} films")

# Sauvegarder
movies_update_df.write_parquet('movies_combined.parquet')


for page in range(1,5):
# Delete the file
  file = Path(f'movies_{page}.parquet')
  if file.exists():
      file.unlink()
      print(f"✓ movies_{page}.parquet deleted")
  else:
      print(f"⚠️ movies_{page}.parquet not found")

Avant dédoublonnage: 226 films
Après dédoublonnage: 226 films
✓ movies_1.parquet deleted
✓ movies_2.parquet deleted
✓ movies_3.parquet deleted
✓ movies_4.parquet deleted


In [ ]:
movies_update_df


watchmode_id,imdb_id,title,year,metascore,genre,plot,nb_sources,sources,prix_min,sources_min
i64,str,str,i64,i32,str,str,i64,list[str],f64,list[str]
1791912,"""tt1493274""","""Caught Stealing""",2025,65,"""Comedy, Crime, Thriller""","""Burned-out ex-baseball player …",10,"[""Amazon"", ""AppleTV""]",24.99,"[""Amazon"", ""AppleTV""]"
1642466,"""tt14961624""","""The Old Guard 2""",2025,44,"""Action, Adventure, Fantasy""","""Andy leads immortal warriors a…",1,"[""Netflix""]",null,null
1756904,"""tt22478818""","""William Tell""",2025,52,"""Action, Biography, Drama""","""The narrative unfolds in the 1…",10,"[""Prime Video"", ""Hoopla"", … ""Amazon""]",6.99,"[""Amazon"", ""AppleTV""]"
1812177,"""tt32119297""","""Blue Sun Palace""",2025,83,"""Drama, Romance""","""When tragedy strikes, an unexp…",5,"[""AppleTV"", ""MUBI""]",4.99,"[""AppleTV""]"
1652893,"""tt17023012""","""The Home""",2025,39,"""Horror, Thriller""","""Follows Max, who realizes that…",13,"[""Amazon"", ""AppleTV"", … ""Hoopla""]",6.99,"[""Plex"", ""Amazon"", ""AppleTV""]"
…,…,…,…,…,…,…,…,…,…,…
1632660,"""tt14127944""","""The Actor""",2025,62,"""Crime, Drama, Mystery""","""When New York actor Paul Cole …",14,"[""Plex"", ""Amazon"", ""AppleTV""]",6.99,"[""Plex"", ""Amazon"", ""AppleTV""]"
1740012,"""tt28613536""","""Bob Trevino Likes It""",2025,70,"""Comedy, Drama""","""When lonely 20-something Lily …",6,"[""Hoopla"", ""AppleTV"", ""FlixFling""]",5.99,"[""AppleTV"", ""FlixFling""]"
1709096,"""tt27714946""","""The Monkey""",2025,62,"""Horror""","""When twin brothers Bill and Ha…",19,"[""Amazon"", ""AppleTV"", … ""YouTube""]",6.99,"[""Plex"", ""Amazon"", … ""Cineplex""]"


In [25]:
old_movies_df = pl.read_csv('movies_complete.csv')
old_movies_df.shape
old_movies_df= old_movies_df.select(['watchmode_id','already_seen','never_watch'])

In [26]:
old_movies_df.columns

['watchmode_id', 'already_seen', 'never_watch']

In [ ]:
# Left join: garde toutes les lignes de movies_updates
# et ajoute A et B de movies_complete quand watchid correspond
movies_df = movies_update_df.join(
    old_movies_df,
    on='watchmode_id',
    how='left'
)


In [ ]:
len(movies_df)

226

In [ ]:
import polars as pl

# Votre DataFrame avec les colonnes
#movies_df = movies_df.with_columns(
#    pl.lit(False).alias('already_seen'),
#    pl.lit(False).alias('never_watch')
#)

In [ ]:


def interactive_tag_movies():
    """Tagger les films interactivement"""
    global movies_df
    
    # Films non encore taggés
    untagged = movies_df.filter(
        (~pl.col('already_seen')) & 
        (~pl.col('never_watch'))
    )
    
    print(f"\n🎬 {len(untagged)} films à tagger\n")
    
    for i, row in enumerate(untagged.iter_rows(named=True), 1):
        imdb_id = row['imdb_id']
        
        print(f"\n{'='*70}")
        print(f"{i}/{len(untagged)} - {row['title']} ({row.get('year', 'N/A')})")
        print(f"⭐ Metascore: {row.get('metascore', 'N/A')}")
        print(f"🎭 Genre: {row.get('genre', 'N/A')}")
        print(f"📖 {row.get('plot', 'Pas de description')}")
        print('='*70)
        
        choice = input("\nAction? (v=vu / j=jamais / s=skip / q=quitter): ").lower()
        
        if choice == 'q':
            print("\n✓ Sauvegarde et sortie...")
            break
        elif choice == 'v':
            # Marquer comme vu
            movies_df = movies_df.with_columns(
                pl.when(pl.col('imdb_id') == imdb_id)
                  .then(True)
                  .otherwise(pl.col('already_seen'))
                  .alias('already_seen')
            )
            print("✓ Marqué comme VU")
        elif choice == 'j':
            # Marquer comme jamais
            movies_df = movies_df.with_columns(
                pl.when(pl.col('imdb_id') == imdb_id)
                  .then(True)
                  .otherwise(pl.col('never_watch'))
                  .alias('never_watch')
            )
            print("🚫 Marqué comme JAMAIS")
        elif choice == 's':
            print("⏭️  Skip")
        else:
            print("❌ Choix invalide, skip")
    
    # Stats finales
    print(f"\n📊 STATISTIQUES:")
    print(f"   Vus: {movies_df['already_seen'].sum()}")
    print(f"   Jamais: {movies_df['never_watch'].sum()}")
    print(f"   Restants: {(~movies_df['already_seen'] & ~movies_df['never_watch']).sum()}")
    
    # Sauvegarder
    movies_df.write_parquet('movies_tagged.parquet')
    print("\n✓ Sauvegardé dans movies_tagged.parquet")

# Lancer
interactive_tag_movies()

In [33]:
titres = movies_df.filter(pl.col('already_seen'))['title'].to_list()

for i, titre in enumerate(titres, 1):
    print(f"{i}. {titre}")

print(f"\nTotal: {len(titres)} films vus")

1. The Accountant²
2. Mickey 17
3. Mission: Impossible - The Final Reckoning
4. Friendship
5. Final Destination Bloodlines
6. Heads of State
7. 40 Acres
8. The Lost Bus
9. Superman
10. Sovereign
11. 28 Years Later
12. Deep Cover
13. Death of a Unicorn
14. Novocaine
15. Last Breath
16. Black Bag
17. The Pickup
18. Nonnas
19. Drop
20. Warfare
21. The Gorge
22. Companion
23. Nobody 2
24. Sinners
25. Ice Road: Vengeance
26. Sharp Corner

Total: 26 films vus


In [34]:
from datetime import datetime
# Get only the date
today_date = datetime.now().date()
print(today_date)  # 

2025-10-13


In [35]:
# Convertir toutes les colonnes List automatiquement
movies_csv = movies_df.with_columns([
    pl.col(col).list.join(', ') if movies_df[col].dtype == pl.List else pl.col(col)
    for col in movies_df.columns
])

movies_csv.write_csv(f'movies_complete_{today_date}.csv')
print("✓ CSV créé sans nested data")

✓ CSV créé sans nested data


# maintenant omdpapi

In [37]:
import polars as pl

# 1. Augmenter la largeur d'affichage
pl.Config.set_tbl_width_chars(1000)
pl.Config.set_fmt_str_lengths(100)

# 2. Afficher toutes les colonnes
pl.Config.set_tbl_cols(-1)

# 3. Afficher plus de lignes
pl.Config.set_tbl_rows(50)

# Maintenant affiche ton DataFrame
print(movies_df)

shape: (226, 13)
┌──────────────┬────────────┬───────────────────────────────────────────┬──────┬───────────┬───────────────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬─────────────────────────────────────────┬──────────┬────────────────────────────────────┬──────────────┬─────────────┐
│ watchmode_id ┆ imdb_id    ┆ title                                     ┆ year ┆ metascore ┆ genre                         ┆ plot                                                                                                  ┆ nb_sources ┆ sources                                 ┆ prix_min ┆ sources_min                        ┆ already_seen ┆ never_watch │
│ ---          ┆ ---        ┆ ---                                       ┆ ---  ┆ ---       ┆ ---                           ┆ ---                                                                                                   ┆ ---        ┆ ---                